Применение и последующее сравнение различных методов машинного обучения в задаче предсказания температуры плавления халькогенидных стекол в зависимости от их элементного состава

In [13]:
# Объявляю нужные библиотеки
import pandas as pd #Библиотека для работы с фреймворками

Работа с исходными данными, взятыми из научной статьи *S. Singla et al. (2023)* [<u>https://doi.org/10.1088/2515-7639/acc6f2</u> ; <u>https://github.com/M3RG-IITD/chgs-ai</u>]. В качестве объекта выступает температура плавления халькогенидных стекол. Признаками же является молярное содержание тех или иных атомов халькогенов.

In [ ]:
# Загрузка датасета из csv-файла
df = pd.read_csv('TG.csv', sep=',')


,Al,As,B,Ba,Be,Bi,C,Ca,Cd,Ce,...,Ta,Tb,Te,Ti,Tl,W,Y,Zn,Zr,TG
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,304.00
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,356.15
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,348.15
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,38.0,0.0,53.0,0.0,0.0,0.0,0.0,353.15
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,32.0,0.0,58.0,0.0,0.0,0.0,0.0,353.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9191,87.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,554.00
9192,87.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,533.00
9193,87.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,556.00
9194,87.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,550.00
